# Exercise 6: PyTorch Introduction with Logistic Regression

## 1 Setup: Install dependencies and download data
- Download the image by running the next cell

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
!pip install -q torch torchvision matplotlib tqdm gdown

import gdown
import os

if not os.path.exists("/content/drive/My Drive/IPEO/Sentinel_2_LakeGeneva.png"):
    gdown.download(id="1U2qBQMfF4E9ZJX7hAl6PmGueYkZrUcuU", output="/content/drive/My Drive/IPEO/Sentinel_2_LakeGeneva.png", quiet=True)

- Alternatively, download the images from [here](https://drive.google.com/file/d/1U2qBQMfF4E9ZJX7hAl6PmGueYkZrUcuU/view?usp=sharing)

## 2. Numpy to Pytorch

**Pytorch is numpy with gradients** and syntax and handling of variables is designed to be very similar.


We start with loading an image with Pillow, convert it to numpy, and then to torch

In [ ]:
from PIL import Image

# Pillow Image (we are not interested in the alpha channel, thats why we convert to RGB only)
image = Image.open('/content/drive/My Drive/IPEO/Sentinel_2_LakeGeneva.png').convert('RGB')

image # jupyter plots a pillow image when it is the last line in the cell

In [ ]:
import numpy as np

# TODO convert the pillow image to numpy array with np.array
numpy_array_image = np.array(image)

# TODO display the values of the array
numpy_array_image

### Conversion numpy to torch tensor

and compare the results

In [ ]:
import torch

# TODO convert the numpy array to a pytorch tensor 
# by calling torch.tensor(nparray) or torch.from_numpy(nparray)
torch_tensor_image = torch.tensor(numpy_array_image)

# TODO print tensor
print(torch_tensor_image)

# TODO bring tensor back to numpy by calling .numpy()
numpy_array_image = torch_tensor_image.numpy()

# TODO print numpy array
print(numpy_array_image)


### Casting to different datatypes

outputs should look like
```
torch.uint8
torch.int64
torch.float32
torch.float64
```

In [ ]:
# TODO print datatype of torch_tensor_image: syntax: tensor.dtype
print(torch_tensor_image.dtype)

# TODO cast the integer image to 32bit integer numbers by calling .long() and print the datatype
print(torch_tensor_image.long().dtype)

# TODO cast the integer image to 32bit floating point numbers by calling .float() and print the datatype
print(torch_tensor_image.float().dtype)

# TODO cast the integer image to 64bit floating point numbers by calling .double() and print the datatype
print(torch_tensor_image.double().dtype)

### Gradients

lets calculate some gradients on the sum function $S = \sum_i a_i$, with $a_i$ as single pixel in the image $A$

**TODO** what is the gradient of the sum $S$ with respect to an individual summands $a_i$:

$\frac{\partial S}{\partial a_i} = 1$

Hint: ask us if you need a hint (it may be simpler than you think). Also, the next cell will output the solution if you implemented it correctly

Side note: If you receive a `UserWarning: To copy construct from a tensor ...`, dont worry about it. 
We want to demonstrate the `requires_grad=True` option. The computationally efficient way `tensor_with_grad = tensor_without_grad.clone().detach().requires_grad_(True)` is not very intuitive

In [ ]:
# TODO check if the torch_tensor_image requires gradients
print(torch_tensor_image.requires_grad)

# TODO cast torch_tensor_image to float (only floating point tensor can have gradients) 
torch_tensor_image_float = torch_tensor_image.float()

# TODO initialize a tensor with gradients (set requires_grad option to True)
torch_tensor_image_with_grad = torch.tensor(torch_tensor_image_float.clone().detach(), requires_grad=True)

# TODO check if torch_tensor_image_with_grad now has gradients enabled
print(torch_tensor_image_with_grad.requires_grad)

# just a rename to be consistent with equation 
A = torch_tensor_image_with_grad

# TODO sum overa all values in the A tensor
mu = A.sum()

# TODO calculate gradients by calling .backward()  
mu.backward()

# TODO print gradients with .grad 
print(A.grad)

## 3 Logistic Regression with Stochastic Gradient Descent

lets first select some training data for classes `land` and `water`

### 3.1 Generate training data

we need to get training data describing the classes `land` and `water`, we do this by 
having input features `x = (green, red, nir)` and labels `y = 0` for `land` and `y = 1` for `water`

In [ ]:
%matplotlib inline
#If plot is not showing, try uncomment the command below
#%notebook inline

import matplotlib.pyplot as plt

image = Image.open('/content/drive/My Drive/IPEO/Sentinel_2_LakeGeneva.png').convert("RGB")

# lets convert to a tensor image with floating point numbers [0,1]
image = torch.from_numpy(np.array(image) / 255.)

# TODO select five training points for each class from the pixels coordinates in the image
water_coords = torch.tensor([(500, 1750), (1000,1000), (1500, 750), (2500, 1000), (2000,800)])
land_coords = torch.tensor([(200, 250), (500, 750), (1000, 2000), (2500, 1750), (2500,500)])

fig, ax = plt.subplots()
ax.imshow(image)
ax.plot(water_coords[:,0], water_coords[:,1], "r*")
ax.plot(land_coords[:,0], land_coords[:,1], "wo")
ax.legend(["water","land"])

In [ ]:
# here, we extract the pixel values for each pixel annotated as water and store it into a water_pixels tensor
water_pixels = []
for point in water_coords:
    r,c = point
    pixel_values = image[c,r]
    water_pixels.append(pixel_values)
water_pixels = torch.stack(water_pixels)

# TODO: do the same for land_pixels
land_pixels = []
for point in land_coords:
    r,c = point
    pixel_values = image[c,r]
    land_pixels.append(pixel_values)
land_pixels = torch.stack(land_pixels)

lets now combine the water and land features with labels and store them 

into 
* `x = [(g1,r1,nir1), (g2,r2,nir2), ...]` (input) and 
* `y = [1,1,1,...,0,0,0,...]` (target/ground truth) 

variables

In [ ]:
# TODO torch.vstack (vertical stack) water pixels and land pixels to a single tensor of size (N x 3)
x = torch.vstack([water_pixels, land_pixels])

# TODO convert x to .float()
x = x.float()

# TODO generate labels of size (N) with zeros for land pixels and ones for water pixels
y_water = torch.ones(water_pixels.shape[0])
y_land = torch.zeros(land_pixels.shape[0])

y = torch.hstack([y_water, y_land])
print(y.shape)
y

### 3.2 Plotting the Feature Space

In [ ]:
%matplotlib inline
from mpl_toolkits import mplot3d

bands = np.array(["green", "red", "nir"])
x_idx = 0
y_idx = 1

classes = np.array(["water", "land"])

fig = plt.figure()
ax = plt.axes(projection='3d')

scatter = ax.scatter(x[:,0], x[:,1], x[:,2], c=y, cmap="RdBu")
el, annot = scatter.legend_elements()
legend1 = ax.legend(el, classes,
                    loc="lower left", title="Classes")
ax.add_artist(legend1)
ax.set_xlabel(f"pixel value {bands[0]} band")
ax.set_ylabel(f"pixel value {bands[1]} band")
ax.set_zlabel(f"pixel value {bands[2]} band")
ax.set_title(f"feature space: green - red - nir")

### 3.3 Implement linear logistic regression classifier

$y = \sigma(\mathbf{Ax} + b)$

that consists of two steps:
1. a linear decision plane $\mathbf{Ax} + b$
2. a sigmoid activation function $\sigma(z) = \frac{1}{1+\exp(-z)}$

#### 3.3.1 Matrix Multiplication and Sigmoid

**TODO** implement step 1: the linear decision plane with random `A` and `b` and our data `x`

we use `nn.Parameter` instead of `torch.Tensor` as they have gradients enabled natively and represent trainable weights

In [ ]:
from torch import nn
torch.random.manual_seed(0) # we set a random seed to have identical "random" outputs

in_features = 3 # RGB
out_features = 1 # 1 probability

# TODO randomly initialize the learnable parameters
A = nn.Parameter(torch.rand([in_features, out_features]))
b = nn.Parameter(torch.rand([out_features]))

# TODO implement Ax + b and store into h
# (you can use the identical torch.matmul(A,B), torch.mm, 
# or "A @ B" for the matrix multiplication)
h = x @ A + b

h # prints h, it represents the distance of each point from this (randomly initialized) decision plane

**TODO** implement step 2, the sigmoid function $\sigma(z) = \frac{1}{1+\exp(-z)}$

Note, you can use torch.exp(tensor) to implement the exponential

In [ ]:
def sigmoid(z):
    return 1 / (1 + torch.exp(-z))

# this output represents the probability of each point. 
# Points on one side will have a positive probability, 
# points on the other a negative.
print(sigmoid(torch.tensor(-100)))
print(sigmoid(torch.tensor(-1)))
print(sigmoid(torch.tensor(0)))
print(sigmoid(torch.tensor(1)))
print(sigmoid(torch.tensor(100)))

let's run the sigmoid function on `h` as the output of $\mathbf{Ax} + b$
this gives us a complete linear layer $\sigma(\mathbf{Ax} + b)$

In [ ]:
sigmoid(h)

#### 3.3.2 Package your code into a Torch Module

FYI: background on Python classes
* the module is a class called `Linear` that 
* has a constructor `__init__` and a member function `forward`
* `self` references the instance of this class and `self.A = ...` initializes a variable of this instance.
* It inherits from the class `nn.Module` (`super().__init__()` is the constructor of the parent class)

**TODO** implement the linear decision plane as `nn.Module` and initialize the `nn.Parameter` `A` and `b` randomly with `torch.rand`

the final output should be identical to `sigmoid(h)` from above

In [ ]:
from torch import nn
torch.random.manual_seed(0) # we set a random seed to have identical "random" outputs

class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        # TODO initialize the learnable parameters A and b with random values of the correct shape
        self.A = nn.Parameter(torch.rand([in_features, out_features]))
        self.b = nn.Parameter(torch.rand([out_features]))
        
        
    def forward(self, x):
        # TODO implement the multiplication Ax + b here
        return torch.matmul(x, self.A) + self.b

# nothing to do here, but note that we need to package sigmoid into 
# a nn.Module as well for nn.Sequential in the next cell
class Sigmoid(nn.Module):
    def forward(self, x):
        return sigmoid(x)
    
    
in_features = 3 # RGB
out_features = 1 # 1 probability

linear_module = Linear(in_features,out_features) # calls __init__() of Linear
sigmoid_function = Sigmoid()

h = linear_module(x) # calls forward(x)
sigmoid_function(h) # should be similar to the previous cell

#### 3.3.3 Single Logistic Regression Model

Lets build and initialize a single `model` that converts pixel vales `x` into class probabilities `y`by chaining `Linear` and `sigmoid` together with `nn.Sequential`


In [ ]:
from collections import OrderedDict
model = nn.Sequential(
        OrderedDict(
            {"linear": Linear(in_features,out_features), 
            "sigmoid": Sigmoid()}
            )
        )

model

#### 3.3.3.4 Optimization with Gradient Descent

In the next cell you will implement the gradient descent

$A \leftarrow A - \eta \frac{\partial e}{\partial A}$, and $b \leftarrow b - \eta \frac{\partial e}{\partial A}$

of parameters `A`, `b` with `error` $e$ and learning rate $\eta$

Note: execute the previous cell to restart from a random model. The expected result is a plot of decreasing training error over time (epochs)

In [ ]:
from tqdm.auto import tqdm

# number of epochs (times the model sees the training dataset)
num_epochs = 2000
learning_rate = 1

errors = []
with tqdm(range(num_epochs)) as progress_bar:
    for epoch in progress_bar:
        probability = model(x)

        # Binary Cross Entropy (BCE) error function
        error = torch.nn.functional.binary_cross_entropy(probability, y.unsqueeze(-1).float())

        # gradient back propagation (built into Torch)
        error.backward()

        # TODO implement gradient descent for A and b: e.g. A = A - lr * A.grad 
        # note that this needs to be wrapped in nn.Parameter
        model.linear.A = nn.Parameter(model.linear.A - learning_rate * model.linear.A.grad)
        model.linear.b = nn.Parameter(model.linear.b - learning_rate * model.linear.b.grad)

        errors.append(error.detach().numpy())
        progress_bar.set_description(f"epoch {epoch} BCE error: {error:.2f}")
        
%matplotlib inline
fig, ax = plt.subplots(figsize=(6,2))
ax.plot(np.arange(num_epochs), errors)
ax.set_xlabel("epoch")
ax.set_ylabel("training error")

### 3.3.4 Test Trained Model

now we apply the trained model on all pixels of the image

to do so, we need to 
1. reshape (flatten) the image of size (H,W,C=3) to a list of pixels (HW,C=3)
2. estimate a probability for each pixel
3. reshape back the probabilities of shape (HW,1) to a probability image (H,W)

**TODO** implement these steps

final expected output is
```
torch.Size([2232, 2998, 3])
torch.Size([6691536, 3])
torch.Size([6691536, 1])
torch.Size([2232, 2998])
```

In [ ]:
image = Image.open('/content/drive/My Drive/IPEO/Sentinel_2_LakeGeneva.png').convert("RGB")
image = torch.from_numpy(np.array(image) / 255.).float()

print(image.shape)
H,W,C = image.shape # stores original sizes in the variables height H, width W, and channels C

# TODO reshape the image to a list of pixels with .reshape 
image_pixels = image.reshape(H*W,C)
print(image_pixels.shape)

# TODO predict a probability for each pixel
pixels_probabilities = model(image_pixels)
print(pixels_probabilities.shape)

# TODO .reshape the list of probabilities (HW,1) to (H,W)
probability_image = pixels_probabilities.reshape(H,W)
print(probability_image.shape)

### Congratulations! You reached the end of the exercise

**no more TODOs here execute the next cells for plotting the results**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(figsize=(12,12))
im = ax.imshow(probability_image.detach().numpy(), vmin=0, vmax=1, cmap="viridis")
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical', label="probability water")
plt.show()

#### Visualization of the pixels in the feature space and and learned decision plane

This cell is for your interest. Nothing to do here.

In [ ]:
%matplotlib inline
from mpl_toolkits import mplot3d
import matplotlib.cm as cm

fig = plt.figure()
ax = plt.axes(projection='3d')

# select only a random subset of pixels for plotting
idxs = np.random.randint(len(image_pixels), size=1000)
colors = cm.viridis(pixels_probabilities[idxs].detach().numpy())

#scatter = ax.scatter(image_pixels[idxs,0], image_pixels[idxs,1], image_pixels[idxs,2], c=image_pixels[idxs])
scatter = ax.scatter(image_pixels[idxs,0], image_pixels[idxs,1], image_pixels[idxs,2], c=colors)

# define decision boundary using A, b
point  = np.array([0, 0, 0])
normal = model.linear.A.detach().squeeze().cpu().numpy()
d = -point.dot(normal) + model.linear.b.detach().squeeze().cpu().numpy()
xx, yy = np.meshgrid(np.linspace(0,1,2), np.linspace(0,1,2))
z = (-normal[0] * xx - normal[1] * yy - d) * 1. /normal[2]

# plot decision boundary
ax.plot_surface(xx, yy, z, alpha=0.5, color="#0065bd")##0065bd

# definition of axis and labels
ax.set_xlabel(f"pixel value {bands[0]} band")
ax.set_ylabel(f"pixel value {bands[1]} band")
ax.set_zlabel(f"pixel value {bands[2]} band")
ax.set_title(f"feature space: green - red - nir")
